In [1]:
!pip install -q transformers==4.7.0 fugashi ipadic
from transformers import BertJapaneseTokenizer, BertModel
import torch


  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\tokenizers
      copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers
      creating build\lib.win-amd64-cpython-311\tokenizers\models
      copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\models
      creating build\lib.win-amd64-cpython-311\tokenizers\decoders
      copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\decoders
      creating build\lib.win-amd64-cpython-311\tokenizers\normalizers
      copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\normalizers
      creating build\lib.

In [2]:
class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            # encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
            #                                truncation=True, return_tensors="pt").to(self.device)
            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="max_length", max_length=512,
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)


In [3]:
model = SentenceBertJapanese("sonoisa/sentence-bert-base-ja-mean-tokens")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'BertJapaneseTokenizer'.


In [4]:
import numpy as np

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [5]:
movie_a = "ゴッドファーザー"
movie_a_comment_1 = "在米イタリア人による，..........皮肉な話だ。"

movie_a_comment_2 = "華やかな音楽も届かない書斎では、.....感じるからだろう。"
movie_a_comment_3 = "まず、こういう映画を毛嫌いする方もいると思いますし、.....と個人的に思っています。"
movie_a_vectors = model.encode([movie_a_comment_1, movie_a_comment_2, movie_a_comment_3])
movie_a_avg_vector = (movie_a_vectors[0].numpy() + movie_a_vectors[1].numpy() + movie_a_vectors[2].numpy()) / 3
                                                    
movie_b = "スパイダーマン"
movie_b_comment_1 = "映画「スパイダーマン」は、他のアメコミ映画とは明らか....のは俺だけではあるまい。"
movie_b_comment_2 = "この映画の魅力…それは.....過ぎないのだから。"
movie_b_comment_3 = "※注意。....、いやなんとなく。"
movie_b_vectors = model.encode([movie_b_comment_1, movie_b_comment_2, movie_b_comment_3])
movie_b_avg_vector = (movie_b_vectors[0].numpy() + movie_b_vectors[1].numpy() + movie_b_vectors[2].numpy()) / 3
                                                    
movie_c = "映画クレヨンしんちゃん　嵐を呼ぶモーレツ！オトナ帝国の逆襲"
movie_c_comment_1 = "子供向けじゃあない....育った大人なんて、ゾっとする。"
movie_c_comment_2 = "今まで観なくて....大事なのかもしれませんね"
movie_c_comment_3 = "ノスタルジーって、....笑えて泣けた。心から！"
movie_c_vectors = model.encode([movie_c_comment_1, movie_c_comment_2, movie_c_comment_3])
movie_c_avg_vector = (movie_c_vectors[0].numpy() + movie_c_vectors[1].numpy() + movie_c_vectors[2].numpy()) / 3


In [6]:
comment = model.encode(["笑って泣ける映画です"])

print(f'vs{movie_a}_平均:{cos_sim(movie_a_avg_vector, comment[0])}')
print(f'vs{movie_a}_コメント1:{cos_sim(movie_a_vectors[0].numpy(), comment[0])}')
print(f'vs{movie_a}_コメント2:{cos_sim(movie_a_vectors[1].numpy(), comment[0])}')
print(f'vs{movie_a}_コメント3:{cos_sim(movie_a_vectors[2].numpy(), comment[0])}')

print(f'vs{movie_b}_平均:{cos_sim(movie_b_avg_vector, comment[0])}')
print(f'vs{movie_b}_コメント1:{cos_sim(movie_b_vectors[0].numpy(), comment[0])}')
print(f'vs{movie_b}_コメント2:{cos_sim(movie_b_vectors[1].numpy(), comment[0])}')
print(f'vs{movie_b}_コメント3:{cos_sim(movie_b_vectors[2].numpy(), comment[0])}')

print(f'vs{movie_c}_平均:{cos_sim(movie_c_avg_vector, comment[0])}')
print(f'vs{movie_c}_コメント1:{cos_sim(movie_c_vectors[0].numpy(), comment[0])}')
print(f'vs{movie_c}_コメント2:{cos_sim(movie_c_vectors[1].numpy(), comment[0])}')
print(f'vs{movie_c}_コメント3:{cos_sim(movie_c_vectors[2].numpy(), comment[0])}')


vsゴッドファーザー_平均:0.4020501673221588
vsゴッドファーザー_コメント1:0.22076258063316345
vsゴッドファーザー_コメント2:0.2961881458759308
vsゴッドファーザー_コメント3:0.39889833331108093
vsスパイダーマン_平均:0.4823598265647888
vsスパイダーマン_コメント1:0.21828965842723846
vsスパイダーマン_コメント2:0.47695136070251465
vsスパイダーマン_コメント3:0.37502574920654297
vs映画クレヨンしんちゃん　嵐を呼ぶモーレツ！オトナ帝国の逆襲_平均:0.6023193001747131
vs映画クレヨンしんちゃん　嵐を呼ぶモーレツ！オトナ帝国の逆襲_コメント1:0.29586368799209595
vs映画クレヨンしんちゃん　嵐を呼ぶモーレツ！オトナ帝国の逆襲_コメント2:0.28229811787605286
vs映画クレヨンしんちゃん　嵐を呼ぶモーレツ！オトナ帝国の逆襲_コメント3:0.7783889770507812
